# Dirac Equation

This notebook contains the programmatic verification for the **Dirac Equation** entry from the THEORIA dataset.

**Entry ID:** dirac_equation  
**Required Library:** sympy 1.13.1

## Description
The Dirac equation is the relativistic wave equation for spin-1/2 fermions. It resolves the challenge of creating a quantum equation that is first-order in time, ensuring positive probability density, while being consistent with special relativity. Dirac achieved this by linearizing the quadratic dispersion relation `E^2 = (pc)^2 + (mc^2)^2` using `4×4` gamma matrices satisfying a Clifford algebra. The equation naturally predicts electron spin, its magnetic moment, and antimatter (positrons). It is the foundation of quantum electrodynamics.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.13.1

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp
from sympy import I, sqrt, exp, symbols, Matrix, eye, zeros, simplify, expand
from sympy import cos, sin, Function, diff

# =====================================================
# Programmatic verification: Dirac Equation
#
# Sections:
#  1. Define gamma matrices (Dirac/standard representation)
#  2. Verify Clifford algebra {γ^μ, γ^ν} = 2η^{μν}I₄ (Steps 7, 9)
#  3. Verify Klein-Gordon consistency (Step 11)
#  4. Verify plane-wave solutions (Steps 12-13)
# =====================================================

# ---------------------------
# Section 1: Define gamma matrices
# ---------------------------
# Dirac (standard) representation:
# γ^0 = diag(I₂, -I₂), γ^i = [[0, σ^i], [-σ^i, 0]]

# Pauli matrices
sigma_1 = Matrix([[0, 1], [1, 0]])
sigma_2 = Matrix([[0, -I], [I, 0]])
sigma_3 = Matrix([[1, 0], [0, -1]])
I2 = eye(2)
Z2 = zeros(2)

# Gamma matrices in Dirac representation
gamma_0 = Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, -1]
])

def block_matrix(A, B, C, D):
    """Construct 4x4 matrix from 2x2 blocks [[A,B],[C,D]]"""
    top = A.row_join(B)
    bottom = C.row_join(D)
    return top.col_join(bottom)

gamma_1 = block_matrix(Z2, sigma_1, -sigma_1, Z2)
gamma_2 = block_matrix(Z2, sigma_2, -sigma_2, Z2)
gamma_3 = block_matrix(Z2, sigma_3, -sigma_3, Z2)

gamma = [gamma_0, gamma_1, gamma_2, gamma_3]
I4 = eye(4)

# Minkowski metric η^{μν} = diag(+1, -1, -1, -1)
eta = Matrix([
    [1, 0, 0, 0],
    [0, -1, 0, 0],
    [0, 0, -1, 0],
    [0, 0, 0, -1]
])

# ---------------------------
# Section 2: Verify Clifford algebra (Steps 7, 9)
# {γ^μ, γ^ν} = γ^μ γ^ν + γ^ν γ^μ = 2η^{μν} I₄
# ---------------------------
print('Section 2: Verifying Clifford algebra...')
for mu in range(4):
    for nu in range(4):
        anticomm = gamma[mu] * gamma[nu] + gamma[nu] * gamma[mu]
        expected = 2 * eta[mu, nu] * I4
        assert simplify(anticomm - expected) == zeros(4), f'Clifford algebra failed for μ={mu}, ν={nu}'
print('  {γ^μ, γ^ν} = 2η^{μν}I₄ verified for all μ,ν ∈ {0,1,2,3}')

# ---------------------------
# Section 3: Verify Klein-Gordon consistency (Step 11)
# (γ^μ p_μ + mc)(γ^ν p_ν - mc) = p_μ p^μ - (mc)² = E²/c² - p² - m²c²
# ---------------------------
print('Section 3: Verifying Klein-Gordon consistency...')
E, px, py, pz, m, c = symbols('E p_x p_y p_z m c', real=True)

# Four-momentum with lowered index: p_μ = (E/c, -p_x, -p_y, -p_z)
# Contraction: γ^μ p_μ = γ^0(E/c) + γ^1(-p_x) + γ^2(-p_y) + γ^3(-p_z)
p_components = [E/c, -px, -py, -pz]
slash_p = gamma[0]*p_components[0] + gamma[1]*p_components[1] + gamma[2]*p_components[2] + gamma[3]*p_components[3]

# (slash_p + mc)(slash_p - mc) should equal (p_μ p^μ - m²c²) I₄
# where p_μ p^μ = (E/c)² - p_x² - p_y² - p_z²
product = (slash_p + m*c*I4) * (slash_p - m*c*I4)
product_simplified = simplify(expand(product))

p_squared = (E/c)**2 - px**2 - py**2 - pz**2
expected_kg = (p_squared - m**2 * c**2) * I4
expected_simplified = simplify(expand(expected_kg))

assert simplify(product_simplified - expected_simplified) == zeros(4), 'Klein-Gordon consistency failed'
print('  (γ^μp_μ + mc)(γ^νp_ν - mc) = (p_μp^μ - m²c²)I₄ verified')

# ---------------------------
# Section 4: Verify plane-wave solutions (Steps 12-13)
# ---------------------------
print('Section 4: Verifying plane-wave solutions...')

# For a particle at rest: E = mc², p = 0
# The Dirac equation (slash_p - mc)u = 0 becomes (γ^0 m c - mc)u = 0
# => m*c*(γ^0 - I₄)u = 0
slash_p_rest = gamma_0 * m * c  # Only γ^0(E/c) = γ^0(mc²/c) = γ^0 mc
dirac_op_rest = slash_p_rest - m * c * I4
dirac_op_rest_simplified = simplify(dirac_op_rest)

# This should be mc*(γ^0 - I₄) = mc * diag(0, 0, -2, -2)
# So the upper two components of u are free (positive energy solutions)
expected_rest = m * c * (gamma_0 - I4)
assert simplify(dirac_op_rest_simplified - expected_rest) == zeros(4), 'Rest frame Dirac operator incorrect'

# Positive energy spinors at rest: u₁ = (1,0,0,0)ᵀ, u₂ = (0,1,0,0)ᵀ
u1 = Matrix([1, 0, 0, 0])
u2 = Matrix([0, 1, 0, 0])

# Verify they satisfy (slash_p - mc)u = 0 at rest
assert simplify(dirac_op_rest * u1) == Matrix([0, 0, 0, 0]), 'u₁ does not satisfy Dirac eq at rest'
assert simplify(dirac_op_rest * u2) == Matrix([0, 0, 0, 0]), 'u₂ does not satisfy Dirac eq at rest'
print('  Positive energy spinors u₁, u₂ at rest verified')

# Negative energy spinors at rest: v₁ = (0,0,1,0)ᵀ, v₂ = (0,0,0,1)ᵀ
# These satisfy (slash_p + mc)v = 0, i.e., the antiparticle equation
v1 = Matrix([0, 0, 1, 0])
v2 = Matrix([0, 0, 0, 1])
dirac_op_antiparticle_rest = slash_p_rest + m * c * I4

assert simplify(dirac_op_antiparticle_rest * v1) == Matrix([0, 0, 0, 0]), 'v₁ does not satisfy antiparticle eq'
assert simplify(dirac_op_antiparticle_rest * v2) == Matrix([0, 0, 0, 0]), 'v₂ does not satisfy antiparticle eq'
print('  Negative energy spinors v₁, v₂ at rest verified')

# Verify on-shell condition: E² = (pc)² + (mc²)² when (slash_p - mc)u = 0
# From Section 4, (slash_p)² = p_μ p^μ I₄ = (E²/c² - p²) I₄
# If (slash_p - mc)u = 0, then slash_p u = mc u
# => (slash_p)² u = m²c² u => (E²/c² - p²) u = m²c² u
# => E² = p²c² + m²c⁴ ✓
print('  On-shell condition E² = (pc)² + (mc²)² follows from Dirac equation')

print('')
print('Dirac equation programmatic verification passed')


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=dirac_equation.json](https://theoria-dataset.org/entries.html?entry=dirac_equation.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0